In [760]:
import numpy as np

In [761]:
import os

In [762]:
from bokeh.io import save, output_file

In [763]:
import pandas as pd

In [764]:
def combine_columns(columns, separator):
    base = np.copy(columns[0].values)
    for c in columns[1:]:
        base += separator + c
    return base

In [765]:
def generate_bokeh_vbar_plot(x_vals, y_vals, bar_labels, title, tooltips=[], use_bar_labels=True):
    from bokeh.io import curdoc
    from bokeh.models import LabelSet, ColumnDataSource, HoverTool, NumeralTickFormatter
    from bokeh.plotting import figure
    # configure theme
    curdoc().theme = 'dark_minimal'
    
    # setup bar labels
    source = ColumnDataSource(data=dict(y_val=y_vals,
                                        x_val=x_vals,
                                        bar_label=bar_labels))
    # create labelset
    labels = LabelSet(x='x_val', y='y_val', text='bar_label', level='glyph', text_align='center', source=source, x_offset=0, y_offset=3, text_color="white")
    f = figure(x_range=x_vals, height=250, title=title, tooltips=tooltips)
    # configure plot
    f.yaxis[0].formatter = NumeralTickFormatter(format="F")
    f.sizing_mode = 'scale_both'
    f.vbar(x='x_val', top='y_val', source=source, width=0.5, hover_fill_color="orange")
    f.xaxis.major_label_orientation = 1.0
    f.x_range.range_padding = 0.0005
    if use_bar_labels:
        f.add_layout(labels)
    return f

In [766]:
spreadsheet_path = "../data/market_03_01_2022.xlsx"

In [767]:
spreadsheet_name = os.path.splitext(os.path.split(spreadsheet_path)[-1])[0]

In [768]:
scrolls_df = pd.read_excel(spreadsheet_path, sheet_name="Scrolls")

In [769]:
scrolls_df

,item,stat,percent,price,owner_name,count,location
0,shield,LUK,10,6666666,sedecreM,1,FM1
1,glove,Att,10,3600000,sedecreM,3,FM1
2,glove,Att,10,3999999,Necrolyptica,1,FM1
3,2h sword,Att,60,299999,Necrolyptica,1,FM1
4,2h sword,Att,10,499999,Necrolyptica,1,FM1
...,...,...,...,...,...,...,...
122,bottom,DEF,10,55555,AVOID,1,FM7
123,helmet,HP,10,99999,AVOID,1,FM7
124,helmet,DEX,60,13999999,AVOID,1,FM7
125,bow,Att,30,11999999,Jwatt,1,FM7


## Create folder for date

In [770]:
date_root_folder_path = f"../out/{spreadsheet_name}"
if not os.path.exists(date_root_folder_path):
    os.mkdir(date_root_folder_path)

In [771]:
# Create combined item name column
scrolls_df["combined_name"] = combine_columns([scrolls_df["item"], scrolls_df["stat"], scrolls_df["percent"].astype(str)], '-')

In [772]:
scrolls_df

,item,stat,percent,price,owner_name,count,location,combined_name
0,shield,LUK,10,6666666,sedecreM,1,FM1,shield-LUK-10
1,glove,Att,10,3600000,sedecreM,3,FM1,glove-Att-10
2,glove,Att,10,3999999,Necrolyptica,1,FM1,glove-Att-10
3,2h sword,Att,60,299999,Necrolyptica,1,FM1,2h sword-Att-60
4,2h sword,Att,10,499999,Necrolyptica,1,FM1,2h sword-Att-10
...,...,...,...,...,...,...,...,...
122,bottom,DEF,10,55555,AVOID,1,FM7,bottom-DEF-10
123,helmet,HP,10,99999,AVOID,1,FM7,helmet-HP-10
124,helmet,DEX,60,13999999,AVOID,1,FM7,helmet-DEX-60
125,bow,Att,30,11999999,Jwatt,1,FM7,bow-Att-30


# Scrolls

In [773]:
scrolls_root_folder = f"{date_root_folder_path}/scrolls"
if not os.path.exists(scrolls_root_folder):
    os.mkdir(scrolls_root_folder)

## Scroll Count

In [774]:
scroll_count_df = pd.DataFrame(scrolls_df["combined_name"].value_counts()).rename_axis("item_name").rename(columns={"combined_name": "count"})
x_labels = np.sort(scroll_count_df.index.values)
y_vals = scroll_count_df.loc[x_labels]["count"].values

In [775]:
scroll_count_plot = generate_bokeh_vbar_plot(x_labels, y_vals, list(map(str, y_vals)), "Scroll Count", [("count", "@y_val"), ("name", "@x_val")])

In [776]:
output_file(f"{scrolls_root_folder}/scroll_count.html")
save(scroll_count_plot)

'F:\\Development\\phoenix-market\\out\\market_03_01_2022\\scrolls\\scroll_count.html'

### =======================================

## Mean Scroll Price

In [777]:
mean_scroll_prices = scrolls_df.groupby(by=["combined_name"]).mean().rename_axis("item_name")
x_labels = np.sort(mean_scroll_prices.index.values)
y_vals = mean_scroll_prices.loc[x_labels]["price"].values

In [778]:
mean_scroll_prices_plot = generate_bokeh_vbar_plot(x_labels, y_vals, list(map(str, y_vals)), "Mean Scroll Price", [("price", "@y_val{F}"), ("name", "@x_val")], use_bar_labels=False)

In [779]:
output_file(f"{scrolls_root_folder}/mean_scroll_prices.html")
save(mean_scroll_prices_plot)

'F:\\Development\\phoenix-market\\out\\market_03_01_2022\\scrolls\\mean_scroll_prices.html'

## Scroll Price By Owner

In [780]:
for scroll_name in scrolls_df["combined_name"].unique():
    scroll_price_by_owner_df = scrolls_df[scrolls_df["combined_name"] == scroll_name]
    sorted_names = sorted(list(zip(scroll_price_by_owner_df["owner_name"].values, scroll_price_by_owner_df["owner_name"].index)), key=lambda x: str.lower(x[0]))
    indexes = [x[1] for x in sorted_names]
    x_labels = [x[0] for x in sorted_names]
    y_vals = scroll_price_by_owner_df.loc[indexes]["price"].values
    plot = generate_bokeh_vbar_plot(x_labels, y_vals, list(map(str, y_vals)), scroll_name, [("price", "@y_val{F}"), ("name", "@x_val")], use_bar_labels=False)
    output_file(f"{scrolls_root_folder}/{scroll_name}_price_by_owner.html")
    save(plot)

ERROR:bokeh.core.validation.check:E-1019 (DUPLICATE_FACTORS): FactorRange must specify a unique list of categorical factors for an axis: duplicate factors found: 'Doodlebob'


## Sold Items Count Per Player

In [781]:
player_root_folder_path = f"{date_root_folder_path}/player"
if not os.path.exists(player_root_folder_path):
    os.mkdir(player_root_folder_path)

In [782]:
item_count_per_player = scrolls_df.groupby(by=["owner_name"]).sum()["count"].sort_values()
x_labels = item_count_per_player.index.values
y_vals = item_count_per_player.values

In [783]:
item_count_per_player_plot = generate_bokeh_vbar_plot(x_labels, y_vals, list(map(str, y_vals)), "Scroll Sell Count Per Player", [("count", "@y_val{F}"), ("name", "@x_val")], use_bar_labels=True)

In [784]:
output_file(f"{player_root_folder_path}/player_items_for_sale_count.html")
save(item_count_per_player_plot)

'F:\\Development\\phoenix-market\\out\\market_03_01_2022\\player\\player_items_for_sale_count.html'